In [ ]:
import numpy as np
from viz import quickplot
import matplotlib.pyplot as plt
import mne
import pickle as pkl
from sim import *
from inverse_solutions import *
from source_covs import *
from util import *
from evaluate import *
%matplotlib qt
pth_res = 'assets'

## Load data

In [ ]:
with open(pth_res + '/leadfield.pkl', 'rb') as file:
    leadfield = pkl.load(file)[0]
with open(pth_res + '/pos.pkl', 'rb') as file:  
    pos = pkl.load(file)[0]
with open(pth_res + '/info.pkl', 'rb') as file:  
    info = pkl.load(file)
chanPos = get_chan_pos_list(info, montage_type='standard_1020')

## Generate random brain activity

In [ ]:
import importlib
import sim
importlib.reload(sim)

settings = {"n_sources": 2,  # number of sources
            "diam": 50,  # diameter of source patches in mm
            "amplitude": 9.5,  # amplitude of source patches
            "shape": 'gaussian',
            "durOfTrial": 1,
            "sampleFreq": 100,
            "snr": 1,
            "filtfreqs": (1, 30),
            "path": 'assets/raw_data',
            "numberOfTrials": 20,
            }

y, simSettings = simulate_source(pos, settings)
y.shape
x = add_real_noise(np.matmul(leadfield, y), settings)

# plt.figure()
# plt.plot(x.T)
# print('')

In [ ]:
settings = {"n_sources": 2,  # number of sources
            "diam": 50,  # diameter of source patches in mm
            "amplitude": 9.5,  # amplitude of source patches
            "shape": 'gaussian',
            "durOfTrial": 1,
            "sampleFreq": 100,
            "snr": 1,
            "filtfreqs": (1, 30),
            "path": 'assets/raw_data',
            "numberOfTrials": 20,
            }

n = 10
sources = [simulate_source(pos, settings) for _ in range(n)]
eegs = [add_real_noise(np.matmul(leadfield, source[0]), settings) for source in sources]

In [ ]:
eegs[0].shape

## Plot True Source

In [ ]:
%matplotlib qt
quickplot(y[:, 50], 'assets', backend='mayavi', title='True Source')
mne.viz.plot_topomap(x[:, 47], pos=info)


## Exhaustive Dipole Search  
The following function tries out each dipole location and picks the one that explains most of the data. This can be considered a brute force approach

In [ ]:
y_est = exhaustive_dipole_search(x, leadfield, pos)
eval_estimate(y, y_est)
auc = auc_eval(y, y_est, plotme=False)
quickplot(y_est, 'assets', backend='mayavi')


# The simplest way to calculate the inverse problem

When formulating the inverse problem, we say that:
```
M = G * J
```
Where...  
**M** is the M/EEG signal vector of size q  
**G** is the leadfield matrix of size q x p  
**J** is the unknown source of size p  
*q*... number of electrodes  
*p*... number of dipoles modelled in the brain  

Using linear algebra, we find J by multiplying the pseudoinverse(^1) of G on both sides  

```
M * pinv(G) = G * pinv(G) * J  
```
since the matrix multiplication of a matrix and its inverse makes itself redundant, we can write:  
```
M * pinv(G) = J
```
and directly computed an inverse solution.  
(^1) We need the pseudoinverse since the normal inverse of a matrix only works for square matrices.

In [ ]:
y_est = np.matmul(np.linalg.pinv(leadfield), x)
quickplot(y_est, pth_res=pth_res, backend='mayavi')
eval_estimate(y, y_est)

# or equivalently:
# y_est = np.linalg.lstsq(leadfield, x)[0]
# quickplot(y_est, 'assets', backend='mayavi')

In that form, however, we have the following problems:

* No noise term
* No covariance matrix in which we can incorporate our prior assumptions

Therefore, we adapt the following term:

```
W = (C * G') * inv(N + ( (G * S) * G' ))
J = W * M
```
Where...  
**W** is the new transformation matrix  
**C** is the source covariance matrix *sourceCov*  
**N** is the noise term *sensorNoise*  

The noise covariance matrix can be regarded as the prior, which in its simplest case is the identitiy matrix **I** (each source is equally likely to be involved). The identity matrix is used for the minimum norm solution.

In [ ]:
sourceCov = np.identity(leadfield.shape[1])  # identity matrix as source covariance
sensorNoise = x * rms(x) * 0.00000000001  # some sensor noise
w = np.matmul( np.matmul(sourceCov, leadfield.T), (np.linalg.inv(sensorNoise + np.matmul(np.matmul(leadfield, sourceCov), leadfield.T))) )
y_est = np.sum(w*x, axis=1)
eval_estimate(y, y_est)
quickplot(y_est, pth_res=pth_res, backend='mayavi', title='Minimum norm solution')

The functions *sourceCovEstimate*, *minimum_norm_estimate* and *loreta* are based on this approach and can be used like this:

In [ ]:
y_loreta = loreta(x, leadfield)
y_mne = minimum_norm_estimate(x, leadfield)
y_mne_reg = minimum_norm_estimate_2(x, leadfield)

y_sourceCov = sourceCovEstimate(x, leadfield, np.random.rand(leadfield.shape[1], leadfield.shape[1]))


In [ ]:
# quickplot(y_sourceCov, pth_res=pth_res, backend='mayavi', title='Random SourceCovariance')
# quickplot(y_mne, pth_res=pth_res, backend='mayavi', title='MNE')
# quickplot(y_mne_reg, pth_res=pth_res, backend='mayavi', title='regularized MNE')
quickplot(y_loreta, pth_res=pth_res, backend='mayavi', title='Loreta')
eval_estimate(y, y_loreta)

We can also use a different prior, e.g. the true source.  
Of course, this is cheating, but interesting nonetheless!  

In [ ]:
# Create Source Cov matrix
sourceCov = np.zeros((leadfield.shape[1], leadfield.shape[1]))
for i in range(leadfield.shape[1]):
    sourceCov[i, i] = y[i]

sensorNoise = x * rms(x) * 0.05  # some sensor noise
# Normalization
sourceCov /= np.max(sourceCov)

w = np.matmul( np.matmul(sourceCov, leadfield.T), (np.linalg.inv(sensorNoise + np.matmul(np.matmul(leadfield, sourceCov), leadfield.T))) )
y_est = np.sum(w*x, axis=1)
quickplot(y_est, pth_res=pth_res, backend='mayavi', title='Inverse solution with true source as prior')
eval_estimate(y, y_est)

# Working through:  
### Spatial fidelity of MEG/EEG source estimates: A general evaluation approach  
Samuelsson, Pele, Mamashli, Ahveninen, Hämäläinen

## MNE:

In [ ]:
# New mne
sensorNoise = np.identity(leadfield.shape[0])  # x * rms(x) * 0.5  # some sensor noise
y_est = minimum_norm_estimate_3(x, leadfield, sensorNoise, tikhonov=1.62)
eval_estimate(y, y_est)
auc = auc_eval(y, y_est, simSettings, pos, plotme=True)
quickplot(y_est, pth_res=pth_res, backend='mayavi', title=f'MNE, auc={auc}')

## sLORETA

In [ ]:
sensorNoise = x * rms(x) * 0.05  # some sensor noise
tikhonov = 0.05
y_est = sloreta(x, leadfield, sensorNoise, tikhonov=tikhonov)
eval_estimate(y, y_est)
auc = auc_eval(y, y_est, simSettings, pos, plotme=True)
quickplot(y_est, pth_res=pth_res, backend='mayavi', title=f'sLORETA, auc={auc}')

## dSPM

In [ ]:
sensorNoise = x * rms(x) * 0.05  # some sensor noise
tikhonov = 0.05
y_est = dspm(x, leadfield, sensorNoise, tikhonov=tikhonov)
eval_estimate(y, y_est)
auc = auc_eval(y, y_est, simSettings, pos, plotme=True)
quickplot(y_est, pth_res=pth_res, backend='mayavi', title=f'dSPM, auc={auc}')

## eLORETA

In [ ]:
np.arange(0.0, 1, 0.01)

In [ ]:
y_est = eloreta(x, leadfield, tikhonov=0.05)
eval_estimate(y, y_est)
auc = auc_eval(y, y_est, simSettings, pos, plotme=False)
quickplot(y_est, pth_res=pth_res, backend='mayavi', title=f'eLORETA, auc={auc}')